# LoVoCCS morphology: Hot ICM, BCG, red sequence, and WL mass position angles

This section of the project <span style="color:red">searches for correlation...</span>

## Main takeaways 

In summary:

* <span style="color:red">We compare...</span>

## Import Statements

In [ ]:
import pandas as pd
pd.set_option('display.max_columns', 500)
import numpy as np
from matplotlib import pyplot as plt
import os

# This adds the directory above to the path, allowing me to import the common functions that I've written in
#  common.py - this just saves me repeating boring code and makes sure its all consistent
import sys
sys.path.insert(0, '../..')
from common import lovoccs_cosmo

## Output paths

Paths to save comparison figures and any output results files - defined here to avoid repetition and make the code easier to read: 

In [ ]:
pa_fig_path = "../../outputs/figures/positions_and_morphology/posang_offsets/"

## Loading data files

Important considerations for this dataset:

* ......

### X-LoVoCCS-I base sample

In [ ]:
xlovoccs_base = pd.read_csv("../../../sample_files/X-LoVoCCSI.csv")
xlovoccs_base.insert(0, 'name', xlovoccs_base['LoVoCCSID'].apply(lambda x: "LoVoCCS-" + str(x)))
xlovoccs_base.head(6)

### X-LoVoCCS-I ICM position angles

### X-LoVoCCS-I ICM morphology metrics

### LoVoCCS-II 

## Comparing ICM and BCG position angles

Previous work (such as the study by [Yuan & Wen, 2022](https://arxiv.org/abs/2209.00204)) has found strong correlations between the alignment (i.e. position angle) of the hot ICM of clusters (measured from X-ray) and their BCGs (measured through optical analysis). We wish to make a similar comparison for our LoVoCCS sample.

### Reading in the BCG candidate sample

As part of sample preparation and research for our analyses, we identified the galaxies that appeared likely to be the BCG of each LoVoCCS galaxy cluster (through manual inspection by experienced cluster scientists), and then extracted the position angle from RCSED (which seems to pull information from HyperLeda for most of our galaxies). In cases where there are multiple BCG-like galaxies, or the BCG is ambiguous, then there may be entries in the 'bcg2' or 'bcg3' columns - it is also possible that a BCG is identified and no position angle is present (**we hope to remedy this in the future**):

In [ ]:
bcg_samp = pd.read_csv("../../sample_files/X-LoVoCCSI_BCG_cands.csv")
# Add a LoVoCCS name column from the ID column so that we can match to the other dataframe
bcg_samp['LoVoCCS_name'] = bcg_samp['LoVoCCSID'].apply(lambda x: "LoVoCCS-" + str(x))
bcg_samp.head(5)

### Joining the measurement samples

The first step in comparing the position angles of the ICM and candidate BCGs is to simply plot the measurements of one against the other - for this to happen we join the two dataframes on the name of the LoVoCCS cluster, that way all the information we require for each cluster is in one row:

In [ ]:
pos_ang_comb = pd.merge(out_df, bcg_samp, on='LoVoCCS_name')
pos_ang_comb.head(5)

### Plotting a one-to-one comparison

Here we plot the set of ICM position angles against the set of BCG position angles. Given the degenerate nature of position angle measurements, we find we need to normalise the rotation angles before plotting them. In cases where there are multiple BCG candidates, we plot the comparison of secondary/tertiary candidate properties against the ICM value in different colour/marker shapes:

In [ ]:
plt.figure(figsize=(6, 6))

plt.minorticks_on()
plt.tick_params(which='both', direction='in', top=True, right=True)

# plt.errorbar(pos_ang_comb['bcg1_pa'].values, pos_ang_comb['position_angle'].values, 
#              yerr=pos_ang_comb[['position_angle-', 'position_angle+']].values.T, fmt='kx', capsize=2)

test_icm_pa = pos_ang_comb['position_angle'].values%360
where_deg = test_icm_pa > 180
test_icm_pa[where_deg] = test_icm_pa[where_deg] - 180

test_bcg_pa = pos_ang_comb['bcg1_pa'].values%360
where_deg = test_bcg_pa > 180
test_bcg_pa[where_deg] = test_bcg_pa[where_deg] - 180

plt.plot(test_bcg_pa, test_icm_pa, 'x', color='black', label="BCG1")

test_bcg_pa = pos_ang_comb['bcg2_pa'].values%360
where_deg = test_bcg_pa > 180
test_bcg_pa[where_deg] = test_bcg_pa[where_deg] - 180

plt.plot(test_bcg_pa, test_icm_pa, 'D', color='darkseagreen', label="BCG2")

test_bcg_pa = pos_ang_comb['bcg3_pa'].values%360
where_deg = test_bcg_pa > 180
test_bcg_pa[where_deg] = test_bcg_pa[where_deg] - 180

if np.nansum(test_bcg_pa) > 0:
    plt.plot(test_bcg_pa, test_icm_pa, 'o', color='slateblue', label="BCG3")

plt.plot([-360, 360], [-360, 360], color='red', linestyle='dashed', label='1:1')

plt.xlim(-20, 200)
plt.ylim(-20, 200)

plt.xlabel(r"BCG Candidate PA [$^{\circ}$]", fontsize=15)
plt.ylabel(r"ICM PA [$^{\circ}$]", fontsize=15)

plt.title("ICM vs BCG Candidate Position Angle", fontsize=16)

plt.legend(loc='best', fontsize=15)
plt.tight_layout()
plt.savefig("../../outputs/figures/positions_and_morphology/icm_bcgcand_pa_comp.pdf")
plt.show()